# Application to Extract Image from Excel

## 1. Import Library

In [32]:
import os
import pandas as pd
from openpyxl_image_loader import SheetImageLoader
from openpyxl import load_workbook
from openpyxl.drawing.image import Image
from PIL import Image
import io
import re
import gc
from concurrent.futures import ThreadPoolExecutor
import time
from io import BytesIO
from openpyxl.utils import column_index_from_string, get_column_letter
from lxml import etree
import zipfile
import shutil
import xml.etree.ElementTree as ET
from PIL import Image as PILImage
from collections import defaultdict

## 2. Application to Extract Images from Excel

### 2.1. Function Codes

In [40]:
def unique_column_names(columns):  # Ensure column names are unique by appending a suffix.
    seen = {}
    new_columns = []
    for col in columns:
        if col in seen:
            seen[col] += 1
            new_columns.append(f"{col}_{seen[col]}")
        else:
            seen[col] = 0
            new_columns.append(col)
    return new_columns

def clean_column_names(columns):  # Standardize column names by capitalizing each word properly.
    cleaned_columns = []
    seen = {}
    for col in columns:
        col = str(col).strip()
        col = " ".join(word.capitalize() for word in col.split())
        if col in seen:
            seen[col] += 1
            col = f"{col} {seen[col]}"
        else:
            seen[col] = 0
        cleaned_columns.append(col)
    return cleaned_columns

def extract_images_from_excel(file_path, output_folder):
    """Extract all images from Excel files with detailed position information."""
    import os
    import re
    import shutil
    import zipfile
    import pandas as pd
    from PIL import Image
    from openpyxl import load_workbook
    from openpyxl.utils import get_column_letter
    from collections import defaultdict
    
    try:
        # Extract filename from path
        file_name_clean = re.search(r'([^\\]+)\.xlsx?$', file_path)
        if file_name_clean:
            file_name_clean = file_name_clean.group(1)
        else:
            file_name_clean = os.path.basename(file_path).replace('.xlsx', '').replace('.xlsm', '')
        
        print(f"Processing file: {file_name_clean}")
        
        # Create a temporary directory for extracting the Excel file
        temp_dir = os.path.join(output_folder, "temp_extraction")
        os.makedirs(temp_dir, exist_ok=True)
        
        # Copy the Excel file to a temporary zip file
        temp_zip = os.path.join(temp_dir, "temp_excel.zip")
        shutil.copy2(file_path, temp_zip)
        
        # Extract the Excel file contents
        with zipfile.ZipFile(temp_zip, 'r') as zip_ref:
            zip_ref.extractall(temp_dir)
        
        # Load the workbook to analyze image positions
        wb = load_workbook(file_path, data_only=True)
        
        # Validate which sheets actually contain images
        sheets_with_images = {}
        for sheet_name in wb.sheetnames:
            sheet = wb[sheet_name]
            if hasattr(sheet, '_images') and sheet._images:
                # Count visible images in the sheet
                visible_images = 0
                for img in sheet._images:
                    if hasattr(img, 'anchor') and (
                        hasattr(img.anchor, 'col') or 
                        hasattr(img.anchor, '_from') or 
                        hasattr(img.anchor, 'to')):
                        visible_images += 1
                
                if visible_images > 0:
                    sheets_with_images[sheet_name] = visible_images
                    print(f"Sheet '{sheet_name}' contains {visible_images} images")
                else:
                    print(f"Sheet '{sheet_name}' has _images attribute but no visible images detected")
            else:
                print(f"Sheet '{sheet_name}' doesn't contain any images")
        
        # Create a mapping of image filenames to their positions
        image_positions = {}
        
        # Store mapping of column indices to column names
        column_name_mapping = {}
        
        # Only use sheets that actually contain images for our mapping
        images_per_sheet = defaultdict(list)
        for sheet_name in sheets_with_images:
            images_per_sheet[sheet_name] = []
        
        # Analyze each sheet for image positions and column names
        for sheet_name in wb.sheetnames:
            sheet = wb[sheet_name]
            
            # Extract column names from the sheet
            # First, we'll try to identify the header row
            header_index = 0  # Default to first row
            header_row_detected = False
            
            # Look for the first row with more than one non-empty cell
            for row_idx in range(1, min(20, sheet.max_row + 1)):  # Check first 20 rows
                non_empty_cells = 0
                for col_idx in range(1, min(10, sheet.max_column + 1)):  # Check first 10 columns
                    if sheet.cell(row=row_idx, column=col_idx).value:
                        non_empty_cells += 1
                
                if non_empty_cells > 1:  # At least two cells with values
                    header_index = row_idx - 1  # -1 because pandas is 0-indexed
                    header_row_detected = True
                    break
            
            # Get all data from the sheet for column processing
            data = []
            for row in sheet.iter_rows(values_only=True):
                data.append(row)
                
            # Convert to DataFrame for easier processing
            df = pd.DataFrame(data)
            
            # Use the header row to set column names
            if len(df) > header_index and header_row_detected:
                df.columns = df.iloc[header_index]
                
                # Process column names using provided functions
                # Remove empty columns
                df = df.dropna(axis=1, how="all")
                
                # Drop "REKAP" section if present
                if hasattr(df.columns, 'str'):
                    df = df.loc[:, ~df.columns.str.contains("REKAP", case=False, na=False)]
                
                # Drop rows before header if header_index + 2 < len(df)
                if header_index + 2 < len(df):
                    df = df.drop(index=list(range(0, header_index + 2))).reset_index(drop=True)
                
                # Ensure column names are unique
                df.columns = unique_column_names(df.columns)
                
                # Normalize column names for consistent detection
                df.columns = df.columns.str.upper().str.strip() if hasattr(df.columns, 'str') else df.columns
                
                # Apply column renaming after creating the DataFrame
                df.columns = clean_column_names(df.columns)
                
                # Remove unwanted "None" and "None" columns
                if hasattr(df.columns, 'str'):
                    df = df.loc[:, ~df.columns.str.match(r"^None$|None*", na=False)]
                
                # Remove " None" from remaining column names
                df.columns = df.columns.str.replace(r"\sNone\b", "", regex=True).str.strip() if hasattr(df.columns, 'str') else df.columns
                
                # Map column indices to cleaned column names
                for col_idx, col_name in enumerate(df.columns):
                    if pd.notna(col_name):
                        column_name_mapping[(sheet_name, col_idx)] = str(col_name)
            
            # Document columns that might contain images
            dokumentasi_cols = []
            if hasattr(df, 'columns') and hasattr(df.columns, 'str'):
                dokumentasi_cols = [col for col in df.columns if isinstance(col, str) and 
                                    ("DOKUMENTASI" in col.upper() or "GAMBAR" in col.upper() or "FOTO" in col.upper())]
            
            # Only process images if the sheet has any
            if not hasattr(sheet, '_images') or sheet_name not in sheets_with_images:
                continue
                
            for img_idx, img in enumerate(sheet._images):
                # Extract the image reference/ID
                if hasattr(img, 'path'):
                    img_filename = os.path.basename(img.path)
                elif hasattr(img, '_path'):
                    img_filename = os.path.basename(img._path)
                elif hasattr(img, 'ref'):
                    img_filename = img.ref
                else:
                    img_filename = f"image{img_idx}"
                
                # Add to images for this sheet
                images_per_sheet[sheet_name].append(img_filename)
                
                # Get the image position information from the anchor
                if hasattr(img, 'anchor'):
                    # There are different types of anchors in Excel
                    # Try to get the most precise position information
                    col_idx = None
                    row_idx = None
                    
                    # OneCell anchor type
                    if hasattr(img.anchor, 'col') and hasattr(img.anchor, 'row'):
                        col_idx = img.anchor.col
                        row_idx = img.anchor.row
                    
                    # TwoCell anchor type (from coordinates)
                    elif hasattr(img.anchor, '_from') and hasattr(img.anchor._from, 'col'):
                        col_idx = img.anchor._from.col
                        row_idx = img.anchor._from.row
                    
                    # TwoCell anchor type (to coordinates)
                    elif hasattr(img.anchor, 'to') and hasattr(img.anchor.to, 'col'):
                        col_idx = img.anchor.to.col
                        row_idx = img.anchor.to.row
                        
                    # If we found position information
                    if col_idx is not None and row_idx is not None:
                        # Get column letter (A, B, C, etc.)
                        col_letter = get_column_letter(col_idx + 1)  # +1 because openpyxl is 0-indexed
                        
                        # Get column name using our mapping
                        col_name = column_name_mapping.get((sheet_name, col_idx), None)
                        
                        # If we don't have a name from our mapping, try to find closest header
                        if not col_name:
                            # Try to find the closest header for this column
                            # Scan up to 5 columns left and right
                            for offset in range(1, 6):
                                # Check columns to the left
                                if col_idx - offset >= 0:
                                    col_name = column_name_mapping.get((sheet_name, col_idx - offset), None)
                                    if col_name and any(keyword in col_name.upper() for keyword in 
                                                     ["DOKUMENTASI", "GAMBAR", "FOTO", "IMAGE", "PICTURE"]):
                                        break
                                
                                # Check columns to the right
                                if col_idx + offset < len(df.columns) if hasattr(df, 'columns') else 0:
                                    col_name = column_name_mapping.get((sheet_name, col_idx + offset), None)
                                    if col_name and any(keyword in col_name.upper() for keyword in 
                                                     ["DOKUMENTASI", "GAMBAR", "FOTO", "IMAGE", "PICTURE"]):
                                        break
                        
                        # If still no column name, use the letter
                        if not col_name:
                            col_name = f"Column_{col_letter}"
                        
                        # Clean up the column name for filename
                        safe_col_name = re.sub(r'[\\/*?:"<>|]', '_', str(col_name))
                        
                        # Store the position information
                        image_positions[img_filename] = {
                            'sheet': sheet_name,
                            'column_letter': col_letter,
                            'column_name': safe_col_name,
                            'row': row_idx + 1,  # +1 because openpyxl is 0-indexed
                        }
        
        # Check if there's a media folder with images
        media_dir = os.path.join(temp_dir, 'xl', 'media')
        if not os.path.exists(media_dir):
            print("No media directory found in the Excel document")
            return False
            
        media_files = os.listdir(media_dir)
        if not media_files:
            print("No media files found in the Excel document")
            return False
            
        print(f"Found {len(media_files)} media files to extract")
        
        # For images without sheet assignment, check if they're assigned to any sheet
        unassigned_images = []
        for img_file in media_files:
            assigned = False
            for sheet_images in images_per_sheet.values():
                if img_file in sheet_images:
                    assigned = True
                    break
            if not assigned:
                unassigned_images.append(img_file)
        
        print(f"Found {len(unassigned_images)} unassigned images")
        
        # Distribute unassigned images to sheets that actually have images
        valid_sheets = list(sheets_with_images.keys())
        if unassigned_images:
            if valid_sheets:
                for i, img_file in enumerate(unassigned_images):
                    sheet_idx = i % len(valid_sheets)
                    sheet_name = valid_sheets[sheet_idx]
                    images_per_sheet[sheet_name].append(img_file)
                    print(f"Assigned unassigned image {img_file} to sheet '{sheet_name}'")
            else:
                # If no valid sheets, create a default sheet category
                default_sheet = "Unassigned"
                images_per_sheet[default_sheet] = unassigned_images
                print(f"No sheets with images found, assigned {len(unassigned_images)} images to '{default_sheet}' category")
        
        # Track images by sheet for logging
        sheet_image_counts = defaultdict(int)
        
        # Analyze image patterns to improve column associations
        # Look for patterns like consecutive images in a column
        image_patterns = {}
        
        # For each sheet, analyze image positions
        for sheet_name, images in images_per_sheet.items():
            rows_with_images = {}
            
            # Group images by row
            for img_file in images:
                if img_file in image_positions:
                    pos = image_positions[img_file]
                    row = pos['row']
                    if row not in rows_with_images:
                        rows_with_images[row] = []
                    rows_with_images[row].append(img_file)
            
            # Find dokumentasi columns
            dok_columns = []
            for (s_name, col_idx), col_name in column_name_mapping.items():
                if s_name == sheet_name and any(keyword in col_name.upper() for keyword in 
                                             ["DOKUMENTASI", "GAMBAR", "FOTO", "IMAGE", "PICTURE"]):
                    dok_columns.append((col_idx, col_name))
            
            # Store pattern info
            image_patterns[sheet_name] = {
                'rows_with_images': rows_with_images,
                'dokumentasi_columns': dok_columns
            }
        
        # Summary of image assignment
        print("\nImage assignment summary:")
        for sheet_name, images in images_per_sheet.items():
            print(f"Sheet '{sheet_name}': {len(images)} images")
        
        # Extract all images with position information when available
        successful_images = 0
        
        # For each image in the media folder
        for img_file in media_files:
            try:
                img_path = os.path.join(media_dir, img_file)
                if not os.path.exists(img_path):
                    continue
                
                # Check if we have position info for this image
                position_info = image_positions.get(img_file, None)
                
                if position_info:
                    # Use detailed position information
                    sheet_name = position_info['sheet']
                    col_letter = position_info['column_letter']
                    col_name = position_info['column_name']
                    row_num = position_info['row']
                    
                    safe_sheet_name = re.sub(r'[\\/*?:"<>|]', '_', sheet_name)
                    
                    output_img_path = os.path.join(
                        output_folder, 
                        f"{file_name_clean}_Sheet_{safe_sheet_name}_Column_{col_name}_Row_{row_num}.png"
                    )
                    
                    # Track which sheet this image came from
                    sheet_image_counts[sheet_name] += 1
                    
                else:
                    # Enhanced fallback - try to estimate position based on patterns
                    
                    # Find which sheet this image belongs to
                    sheet_name = "Unknown"
                    found_sheet = False
                    for s_name, sheet_images in images_per_sheet.items():
                        if img_file in sheet_images:
                            sheet_name = s_name
                            found_sheet = True
                            break
                    
                    if not found_sheet:
                        # If we can't determine the sheet, try to use the first sheet that actually has images
                        if valid_sheets:
                            sheet_name = valid_sheets[0]
                            print(f"Image {img_file} not found in any sheet mapping, using sheet '{sheet_name}'")
                        else:
                            print(f"Warning: Image {img_file} not found in any sheet mapping, and no sheets with images detected")
                    
                    safe_sheet_name = re.sub(r'[\\/*?:"<>|]', '_', sheet_name)
                    
                    # Try to find a logical position based on patterns
                    est_column = "Unknown"
                    est_row = 1
                    
                    # If we have documentation columns for this sheet
                    if sheet_name in image_patterns and image_patterns[sheet_name]['dokumentasi_columns']:
                        dok_cols = image_patterns[sheet_name]['dokumentasi_columns']
                        if dok_cols:
                            # Use the first documentation column
                            est_column = re.sub(r'[\\/*?:"<>|]', '_', dok_cols[0][1])
                    
                    # Otherwise, look for any column with image/foto/dokumentasi in the name
                    else:
                        for (s_name, col_idx), col_name in column_name_mapping.items():
                            if s_name == sheet_name and any(keyword in col_name.upper() for keyword in 
                                                         ["DOKUMENTASI", "GAMBAR", "FOTO", "IMAGE", "PICTURE"]):
                                est_column = re.sub(r'[\\/*?:"<>|]', '_', col_name)
                                break
                    
                    # If still no column, use a generic name
                    if est_column == "Unknown":
                        est_column = "Image_Column"
                    
                    # Estimate row based on image index in the sheet
                    if sheet_name in images_per_sheet:
                        img_idx = images_per_sheet[sheet_name].index(img_file) \
                                  if img_file in images_per_sheet[sheet_name] else 0
                        est_row = img_idx + 1
                    
                    output_img_path = os.path.join(
                        output_folder, 
                        f"{file_name_clean}_Sheet_{safe_sheet_name}_Column_{est_column}_Row_{est_row}.png"
                    )
                    
                    sheet_image_counts[sheet_name] += 1
                
                # Save the image
                with Image.open(img_path) as img:
                    img.save(output_img_path, "PNG")
                
                successful_images += 1
                
            except Exception as e:
                print(f"Error processing image {img_file}: {str(e)}")
        
        print(f"✅ Successfully extracted {successful_images} images")
        print("\nImage extraction summary:")
        for sheet_name, count in sheet_image_counts.items():
            print(f"Sheet '{sheet_name}': {count} images extracted")
        
        # Clean up
        wb.close()
        
        # Remove temporary directory
        try:
            shutil.rmtree(temp_dir)
        except Exception as e:
            print(f"Warning: Could not remove temporary directory: {str(e)}")
        
        return True
        
    except Exception as e:
        print(f"Error processing file '{file_path}': {str(e)}")
        import traceback
        traceback.print_exc()
        return False
    finally:
        # Make sure the workbook is closed
        if 'wb' in locals() and wb is not None:
            try:
                wb.close()
            except:
                pass
        
        # Make sure temporary directory is removed if it exists
        if 'temp_dir' in locals() and os.path.exists(temp_dir):
            try:
                shutil.rmtree(temp_dir)
            except:
                pass

def process_excel_folder(folder_path, export_folder):
    """Process all Excel files in a folder and extract all images directly."""
    import os
    
    # Create output folder
    output_folder = os.path.join(export_folder, "Extract Images")
    os.makedirs(output_folder, exist_ok=True)
    
    # Track statistics
    total_files = 0
    successful_files = 0
    failed_files = []
    
    # Get all Excel files in the folder
    excel_files = []
    for file in os.listdir(folder_path):
        if file.endswith(('.xlsx', '.xlsm')):
            excel_files.append(os.path.join(folder_path, file))
    
    if not excel_files:
        print("⚠️ No Excel files found in the specified folder.")
        return
    
    total_files = len(excel_files)
    print(f"🔍 Found {total_files} Excel files to process.")
    print(f"🗂️ All images will be saved to: {output_folder}")
    
    # Process each Excel file
    for i, file_path in enumerate(excel_files, 1):
        file_name = os.path.basename(file_path)
        print(f"\n📊 Processing file {i}/{total_files}: {file_name}")
        
        if extract_images_from_excel(file_path, output_folder):
            successful_files += 1
        else:
            failed_files.append(file_name)
    
    # Print summary
    print("\n" + "="*50)
    print("📈 PROCESSING SUMMARY")
    print("="*50)
    print(f"Total files: {total_files}")
    print(f"Successfully processed: {successful_files}")
    print(f"Failed to process: {len(failed_files)}")
    print(f"Images saved to: {output_folder}")
    
    if failed_files:
        print("\nFiles that could not be processed:")
        for file in failed_files:
            print(f"- {file}")
    
    print("\n🎉 All Excel files processing completed!")

### 2.2. Run Function

In [41]:
excel_folder = r"C:\Users\kanzi\Documents\Part Time Job\Data Hasil Survey"  # Path to Excel files
export_folder = r"C:\Users\kanzi\Documents\Part Time Job\Hasil Export"  # Path for export results
        
# Run the function with your paths
process_excel_folder(excel_folder, export_folder)
#extract_images_from_excel(excel_folder, export_folder)

🔍 Found 2 Excel files to process.
🗂️ All images will be saved to: C:\Users\kanzi\Documents\Part Time Job\Hasil Export\Extract Images

📊 Processing file 1/2: 02. CILEUNGSI - CIBINONG (CITEUREUP).xlsx
Processing file: 02. CILEUNGSI - CIBINONG (CITEUREUP)
Sheet 'RAMBU' contains 94 images
Sheet 'PJU' contains 148 images
Sheet 'RPPJ' contains 14 images
Sheet 'PAGAR PENGAMAN' contains 6 images
Sheet 'MARKA' contains 1 images
Sheet 'WARNING LIGHT' contains 14 images
Sheet 'APILL' doesn't contain any images
Sheet 'ZOSS' contains 1 images
Sheet 'FAS PENYEBERANGAN' contains 8 images
Sheet 'RAMBU PORTABLE' doesn't contain any images
Sheet 'TRAFFIC CONE' doesn't contain any images
Sheet 'WATER BARRIER' doesn't contain any images
Sheet 'CERMIN TIKUNG' contains 6 images
Found 263 media files to extract
Found 262 unassigned images
Assigned unassigned image image10.png to sheet 'RAMBU'
Assigned unassigned image image100.png to sheet 'PJU'
Assigned unassigned image image101.png to sheet 'RPPJ'
Assigned

Traceback (most recent call last):
  File "C:\Users\kanzi\AppData\Local\Temp\ipykernel_17840\2268109085.py", line 58, in extract_images_from_excel
    with zipfile.ZipFile(temp_zip, 'r') as zip_ref:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\kanzi\AppData\Local\Programs\Python\Python312\Lib\zipfile\__init__.py", line 1338, in __init__
    self._RealGetContents()
  File "c:\Users\kanzi\AppData\Local\Programs\Python\Python312\Lib\zipfile\__init__.py", line 1405, in _RealGetContents
    raise BadZipFile("File is not a zip file")
zipfile.BadZipFile: File is not a zip file
